# Comprehensive Demo

## 0. Setup Instructions

**Requirements**
In general, the requirements are similar to Facebook's `maskrcnn-benchmark`:

- Python <= 3.8
- PyTorch >= 1.2
- torchvision >= 0.4
- cocoapi
- yacs
- matplotlib
- GCC >= 4.9
- OpenCV


In [1]:
# Run this setup cell every time this notebook is open or the runtime is reset.
import os

# Set this to True if you are running this notebook in Google Colab
colab = True

if colab:
    # Mount the google drive to access the data
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    folder_path = "/content/drive/Shared drives/COMP 459/" # Path to where you want to store the repository
    %cd "{folder_path}"

    if os.path.isdir("sgg-relevance-scoring"):
        %cd sgg-relevance-scoring
        ! git pull
    else:
        ! git clone https://github.com/duy-ha-dev/sgg-relevance-scoring.git
        %cd sgg-relevance-scoring


Mounted at /content/drive
/content/drive/Shared drives/COMP 459
/content/drive/Shared drives/COMP 459/sgg-relevance-scoring
Already up to date.


## 1. Creating Conda Environment for Python 3.8 (Not Required if Using Colab)

The conda environment that you are planning on using for this setup should be the one running on this notebook. If this is the case, please proceed to the next step; otherwise, in your terminal, you can execute the following commands to set up your environment. **This installation notebook assumes you have a fresh new environment to avoid dependency issues**

`$ conda create --name sgg_relevance_scoring python=3.8`

To activate this new environment:

`$ conda activate sgg_relevance_scoring`

Make sure to deactivate the environment when you are done using it:

`$ conda deactivate`

In [ ]:
if not colab:
    # Confirm that the notebook is running in the correct environment
    !conda info

    # Also confirm that the environment is running on Python 3.8
    !python --version


     active environment : ssg_relevance_scoring
    active env location : C:\Users\maple\anaconda3\envs\ssg_relevance_scoring
            shell level : 1
       user config file : C:\Users\maple\.condarc
 populated config files : C:\Users\maple\.condarc
          conda version : 23.3.1
    conda-build version : 3.23.1
         python version : 3.9.15.final.0
       virtual packages : __archspec=1=x86_64
                          __cuda=12.3=0
                          __win=0=0
       base environment : C:\Users\maple\anaconda3  (writable)
      conda av data dir : C:\Users\maple\anaconda3\etc\conda
  conda av metadata url : None
           channel URLs : https://conda.anaconda.org/menpo/win-64
                          https://conda.anaconda.org/menpo/noarch
                          https://conda.anaconda.org/bioconda/win-64
                          https://conda.anaconda.org/bioconda/noarch
                          https://conda.anaconda.org/r/win-64
                          htt

## 2. Installing Dependencies

### 2.1 Required Python Packages

In [2]:
!python -m pip install ipython
!python -m pip install scipy
!python -m pip install h5py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.4 MB/s eta 0:00:00


### 2.2 Dependencies for maskrcnn_benchmark coco api

In [3]:
!python -m pip install ninja yacs cython matplotlib tqdm opencv-python overrides
!pip install --upgrade setuptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 5.6 MB/s eta 0:00:00
  Using cached setuptools-69.5.1-py3-none-any.whl (894 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2


### 2.3 PyTorch

Please install the correct PyTorch version for your GPU. If your GPU does not support CUDA, then you will have to install CPU version of PyTorch.

Requirements:
- PyTorch >= 1.2.0
- torchvision >= 0.4.0

In [ ]:
# 1. Verify the most recent version of CUDA your GPU supports.
# If this command does not work, you may not have an NVIDIA GPU with CUDA support, in which case you can download CPU version of PyTorch.
!nvidia-smi

Wed Mar 20 22:42:49 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.09                 Driver Version: 546.09       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650 Ti   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   76C    P8               2W /  50W |    221MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

##### 2. Install PyTorch and torchvision following the process on the PyTorch website (https://pytorch.org/get-started/locally/).

Example on your terminal with the correct conda environment activated:

`conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia`

In [2]:
# 3. Verify torch and cuda version
import torch
print(f"Torch version: {torch.__version__}")
print(f"CUDA version: {torch.version.cuda}")

Torch version: 2.2.1+cu121
CUDA version: 12.1


### 2.4 Pycocotools
The git repository is already cloned `dependencies/cocoapi`. You just need to run the `setup.py` in `dependencies/cocoapi/PythonAPI`. There is a separate installation for Windowsl for some reason, coco devs do not want to support Windows.

In [3]:
# Create a dependencies folder for the COCO API and Apex down the line.
!mkdir dependencies

# Clone into dependencies folder
%cd dependencies
if not os.path.isdir("cocoapi"):
    !git clone https://github.com/philferriere/cocoapi.git
    # Can uncomment out below for the original COCO API, but the above also supports Windows.
    # !git clone https://github.com/cocodataset/cocoapi.git

%cd cocoapi/PythonAPI
!make
!make install
!python setup.py install
%cd ../..

/content/drive/Shareddrives/COMP 459/sgg-relevance-scoring/dependencies
Cloning into 'cocoapi'...
remote: Enumerating objects: 994, done.
remote: Total 994 (delta 0), reused 0 (delta 0), pack-reused 994
Receiving objects: 100% (994/994), 12.26 MiB | 12.94 MiB/s, done.
Resolving deltas: 100% (589/589), done.
/content/drive/Shareddrives/COMP 459/sgg-relevance-scoring/dependencies/cocoapi/PythonAPI
python setup.py build_ext --inplace
Compiling pycocotools/_mask.pyx because it changed.
[1/1] Cythonizing pycocotools/_mask.pyx
/usr/local/lib/python3.10/dist-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /content/drive/Shareddrives/COMP 459/sgg-relevance-scoring/dependencies/cocoapi/PythonAPI/pycocotools/_mask.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
../common/maskApi.c: In function ‘rleDecode’:
../common/maskApi.c:46:7: warning: this ‘for’ clause does not 

### 2.5 Apex

The git repository is already cloned in `dependencies/apex`. You just need to run the `setup.py` in `dependencies/apex`:

In [5]:
if not os.path.isdir("apex"):
    !git clone https://github.com/NVIDIA/apex.git

%cd apex
!python setup.py install
%cd ../..

Cloning into 'apex'...
remote: Enumerating objects: 11660, done.
remote: Counting objects: 100% (3728/3728), done.
remote: Compressing objects: 100% (578/578), done.
remote: Total 11660 (delta 3360), reused 3279 (delta 3147), pack-reused 7932
Receiving objects: 100% (11660/11660), 15.48 MiB | 10.88 MiB/s, done.
Resolving deltas: 100% (8189/8189), done.
Updating files: 100% (495/495), done.
/content/drive/Shareddrives/COMP 459/sgg-relevance-scoring/dependencies/apex
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'

 If your intention is to cross-compile, this is not an error.
By default, Apex will cross-compile for Pascal (compute capabilities 6.0, 6.1, 6.2),
Volta (compute capability 7.0), Turing (compute capability 7.5),
and, if the CUDA version is >= 11.0, Ampere (compute capability 8.0).
If you wish to cross-compile for a single specific architecture,
export TORCH_CUDA_ARCH_LIST="compute capability" before running setup.py.



torch.__version__  = 2.2.1+cu121


running in

### 2.6 Install and compile the Scene-Graph Generation Model

In [10]:
%cd scene-graph-benchmark
!python setup.py build develop

/content/drive/Shareddrives/COMP 459/sgg-relevance-scoring/scene-graph-benchmark
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running build
running build_py
creating build
creating build/lib.linux-x86_64-cpython-310
creating build/lib.linux-x86_64-cpython-310/maskrcnn_benchmark
copying maskrcnn_benchmark/__init__.py -> build/lib.linux-x86_64-cpython-310/maskrcnn_benchmark
creating build/lib.linux-x86_64-cpython-310/maskrcnn_benchmark/config
copying maskrcnn_benchmark/config/__init__.py -> build/lib.linux-x86_64-cpython-310/maskrcnn_benchmark/config
copying maskrcnn_benchmark/config/defaults.py -> build/lib.linux-x86_64-cpython-310/maskrcnn_benchmark/config
copying maskrcnn_benchmark/config/paths_catalog.py -> build/lib.linux-x86_64-cpython-310/maskrcnn_benchmark/config
creating build/lib.linux-x86_64-cpython-310/maskrcnn_benchmark/data
copying maskrcnn_benchmark/data/__init__.py -> build/lib.linux-x86_64-cpython-310/maskrcnn_benchmark/data
copying maskrcnn_benchmark/data

## Restart Notebook or Runtime!